In [2]:
import chromadb
import fitz
import spacy

from pprint import pprint
from pathlib import Path
import os
import re
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings
import openai
from pymongo import MongoClient
from chromadb import Client as ChromaClient


In [3]:
# Configurando o ChromaDB com persistência
chroma_client = chromadb.PersistentClient(path="./chroma_db")

In [4]:
# Conexões com o mongodb
mongo_client = MongoClient("mongodb://localhost:27017")
mongo_db = mongo_client["rag_db"]
mongo_chunks = mongo_db["chunks"]

collection = chroma_client.get_or_create_collection(name="rag_embeddings")

In [9]:
import os
import json
import requests
import re
import fitz
import spacy
from datetime import datetime

# Diretório temporário para armazenar PDFs baixados
download_dir = './temp_pdfs/'
os.makedirs(download_dir, exist_ok=True)

# Carrega o modelo e aumenta o limite máximo de caracteres
nlp = spacy.load("pt_core_news_sm")
nlp.max_length = 2_000_000

# Palavras e frases a ignorar
GENERIC_IGNORE_KEYWORDS = {
    "sumário", "índice", "resumo", "anexo", "figura", "tabela", "referência", "bibliografia",
    "conteúdo", "protocolo", "gov", "secretaria", "assinatura", "documento assinado",
    "orientações gerais", "preenchimento", "manual", "parecer técnico"
}

PHRASE_BLACKLIST = [
    r"monitoramento de.*\(e-cenários\)",
    r"^página \d+.*",
    r"^mapas\s*-\s*",
    r"^inserção do estudo.*",
    r"^um arquivo não substitui.*",
    r"^o link para.*tipologia.*",
    r"documentos, manifestações.*",
    r"requerimento.*preenchido.*",
    r"documento.*válido.*"
]

KEY_VERBS = {
    "avaliar", "caracterizar", "delimitar", "analisar", "estimar",
    "descrever", "propor", "identificar", "considerar", "indicar",
    "demonstrar", "impactar", "recomendar", "quantificar"
}

# Função para download de PDF
def download_pdf(url, download_dir):
    response = requests.get(url)
    if response.status_code == 200:
        filename = os.path.join(download_dir, url.split("/")[-1])
        with open(filename, 'wb') as file:
            file.write(response.content)
        return filename
    print(f"Erro ao baixar {url}")
    return None

# Função para verificar a presença de verbos-chave
def contains_key_verb(sent):
    return any(tok.lemma_ in KEY_VERBS for tok in nlp(sent))

# Função para determinar a relevância da linha
def is_line_irrelevant(line):
    lower = line.lower()
    if len(lower.strip()) < 15:
        return True
    if any(k in lower for k in GENERIC_IGNORE_KEYWORDS):
        return True
    if any(re.search(pattern, lower) for pattern in PHRASE_BLACKLIST):
        return True
    if re.search(r"\d{2}/\d{2}/\d{2,4}", lower):
        return True
    if lower.endswith(".pdf"):
        return True
    if re.match(r"^\d+(\.\d+)+\s+", lower):
        return True
    if lower.isupper() and len(lower.split()) > 5:
        return True
    return False

# Função para dividir texto em chunks
def split_text(text, max_chars=500000):
    chunks = []
    while len(text) > max_chars:
        split_point = text[:max_chars].rfind(".") + 1
        if split_point < 50:
            split_point = max_chars
        chunks.append(text[:split_point].strip())
        text = text[split_point:].strip()
    if text:
        chunks.append(text)
    return chunks

# Função para extrair texto relevante
def extract_relevant_sentences(text, min_words=4):
    relevant = []
    chunks = split_text(text)
    for chunk in chunks:
        doc = nlp(chunk)
        for sent in doc.sents:
            s = sent.text.strip()
            if len(s.split()) < min_words:
                continue
            if contains_key_verb(s):
                relevant.append(s)
            elif any(tok.pos_ == "VERB" for tok in sent):
                relevant.append(s)
    return relevant

# Função para extrair texto limpo com spaCy
def extract_clean_text_with_spacy(pdf_path, source_url):
    try:
        with fitz.open(pdf_path) as doc:
            raw_text = "\n".join([page.get_text("text") for page in doc])
        lines = raw_text.splitlines()
        filtered = [line.strip() for line in lines if not is_line_irrelevant(line.strip())]
        cleaned_text = " ".join(filtered)
        relevant_sentences = extract_relevant_sentences(cleaned_text)
        return {
            "source_url": source_url,
            "text": "\n".join(relevant_sentences) if relevant_sentences else None
        }
    except Exception as e:
        print(f"Erro ao processar {pdf_path}: {e}")
        return None

# Lista de URLs
document_urls = [
    "https://cetesb.sp.gov.br/eiarima/eia/EIA-384-24-e-amb-90935-24-Unid-Proc-Oleos-Contam-PCBs-MG-Trafos-Pedreira.pdf"
]

# Processamento dos PDFs
pdf_texts = {}

for url in document_urls:
    print(f"Baixando: {url}...")
    pdf_path = download_pdf(url, download_dir)
    if pdf_path:
        print(f"Processando: {pdf_path}...")
        pdf_data = extract_clean_text_with_spacy(pdf_path, url)
        if pdf_data:
            pdf_texts[os.path.basename(pdf_path)] = pdf_data

print(f"Extração concluída para {len(pdf_texts)} documentos.")


Baixando: https://cetesb.sp.gov.br/eiarima/eia/EIA-384-24-e-amb-90935-24-Unid-Proc-Oleos-Contam-PCBs-MG-Trafos-Pedreira.pdf...
Processando: ./temp_pdfs/EIA-384-24-e-amb-90935-24-Unid-Proc-Oleos-Contam-PCBs-MG-Trafos-Pedreira.pdf...
Extração concluída para 1 documentos.


In [10]:
# Carregar a chave da API do ambiente
openai_api_key = os.getenv("OPENAI_API_KEY")

# Função para dividir os textos em chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings

# Função para dividir os textos em chunks
def split_text_into_chunks(text, chunk_size=512, chunk_overlap=50):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
    )
    return splitter.split_text(text)

# Função para gerar os embeddings de cada chunk de acordo com a formatação da OPENAI
def generate_embeddings(chunks):
    embeddings_model = OpenAIEmbeddings(openai_api_key=openai_api_key)
    return embeddings_model.embed_documents(chunks)

# Gerando os chunks e embeddings para os textos extraídos dos documentos
text_chunks = {}
embeddings = {}

for pdf_file, pdf_data in pdf_texts.items():
    source_url = pdf_data.get("source_url")
    full_text = pdf_data.get("text", "")

    if not isinstance(full_text, str):
        print(f"Aviso: O texto extraído de {pdf_file} não é uma string. Convertendo para string vazia.")
        full_text = ""

    text_chunks[pdf_file] = split_text_into_chunks(full_text)
    embeddings[pdf_file] = generate_embeddings(text_chunks[pdf_file])

    print(f"Texto do arquivo {pdf_file} dividido em {len(text_chunks[pdf_file])} blocos e embeddings gerados.")

print(f"\nOs textos de {len(pdf_texts)} documentos foram particionados e processados em embeddings.")

# Configurando o ChromaDB com persistência
chroma_client = chromadb.PersistentClient(path="./chroma_db")

# Conexões com o MongoDB
mongo_client = MongoClient("mongodb://localhost:27017")
mongo_db = mongo_client["rag_db"]
mongo_chunks = mongo_db["chunks"]

collection = chroma_client.get_or_create_collection(name="rag_embeddings")

# Salvando dados processados no banco de dados
total_chunks = 0

for pdf_file in text_chunks:
    source_url = pdf_texts[pdf_file].get("source_url")
    chunks = text_chunks[pdf_file]
    vector_list = embeddings[pdf_file]

    print(f"Salvando documento: {pdf_file} ({len(chunks)} chunks)")

    for idx, (chunk_text, vector) in enumerate(zip(chunks, vector_list)):
        chunk_id = f"{pdf_file}_chunk_{idx}"

        # Verificar se o chunk já existe no MongoDB
        if mongo_chunks.find_one({"chunk_id": chunk_id}):
            print(f" - Chunk '{chunk_id}' já existe no MongoDB. Pulando.")
            continue

        # Verificar se o ID já existe no ChromaDB
        existing_ids = collection.get(ids=[chunk_id])
        if existing_ids and existing_ids["ids"]:
            print(f" - Chunk '{chunk_id}' já existe no ChromaDB. Pulando.")
            continue

        # Inserir no MongoDB
        mongo_chunks.insert_one({
            "pdf_file": pdf_file,
            "chunk_index": idx,
            "chunk_text": chunk_text,
            "chunk_id": chunk_id,
            "source_url": source_url
        })
        print(f"Embedding para {chunk_id}: {type(vector)} - {vector[:5]}")  # Mostra os primeiros 5 elementos
        # Inserir no ChromaDB
        collection.add(
            ids=[chunk_id],
            embeddings=[vector],
            metadatas=[{
                "chunk_index": idx,
                "source_file": pdf_file,
                "source_url": source_url
            }]
        )

        total_chunks += 1

    print(f" -> Documento '{pdf_file}' salvo com sucesso.\n")

print(f"\n{total_chunks} chunks armazenados no MongoDB e no ChromaDB.")


Texto do arquivo EIA-384-24-e-amb-90935-24-Unid-Proc-Oleos-Contam-PCBs-MG-Trafos-Pedreira.pdf dividido em 573 blocos e embeddings gerados.

Os textos de 1 documentos foram particionados e processados em embeddings.
Salvando documento: EIA-384-24-e-amb-90935-24-Unid-Proc-Oleos-Contam-PCBs-MG-Trafos-Pedreira.pdf (573 chunks)
 -> Documento 'EIA-384-24-e-amb-90935-24-Unid-Proc-Oleos-Contam-PCBs-MG-Trafos-Pedreira.pdf' salvo com sucesso.


573 chunks armazenados no MongoDB e no ChromaDB.


In [6]:
all_embeddings = collection.get()
print(f"Total de embeddings armazenados: {len(all_embeddings['ids'])}")


Total de embeddings armazenados: 573


In [14]:
openai_api_key = os.getenv("OPENAI_API_KEY")
def test_query(query: str):
    # Gerar o embedding da query
    embedding_model = OpenAIEmbeddings(openai_api_key=openai_api_key)
    query_embedding = embedding_model.embed_query(query)
    
    # Executar a consulta no ChromaDB
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=5
    )

    # Verificar se houve retorno
    if not results["metadatas"]:
        print("Nenhum resultado encontrado para a query.")
        return

    # Exibir os resultados
    for idx, metadata in enumerate(results["metadatas"][0]):
        chunk_id = metadata["source_file"] + "_chunk_" + str(metadata["chunk_index"])
        print(f"\nResultado {idx + 1}:")
        print(f"Documento: {metadata['source_file']}")
        print(f"Chunk ID: {chunk_id}")
        print(f"Link: {metadata['source_url']}")

        # Buscar o texto no MongoDB
        chunk = mongo_chunks.find_one({"chunk_id": chunk_id})
        if chunk:
            print(f"Texto: {chunk['chunk_text'][:300]}...")  # Limitar a exibição a 300 caracteres

# Exemplo de consulta
test_query("Quais municipios estão incluidos no APA?")



Resultado 1:
Documento: EIA-384-24-e-amb-90935-24-Unid-Proc-Oleos-Contam-PCBs-MG-Trafos-Pedreira.pdf
Chunk ID: EIA-384-24-e-amb-90935-24-Unid-Proc-Oleos-Contam-PCBs-MG-Trafos-Pedreira.pdf_chunk_269
Link: https://cetesb.sp.gov.br/eiarima/eia/EIA-384-24-e-amb-90935-24-Unid-Proc-Oleos-Contam-PCBs-MG-Trafos-Pedreira.pdf
Texto: Com uma área de 280.330 hectares, a APA inclui total ou parcialmente os municípios paulistas de Amparo, Bragança Paulista, Campinas, Holambra, Jaguariúna, Joanópolis, Mairiporã, Monte Alegre do Sul, Morungaba, Nazaré Paulista, Pedra Bela, Pedreira, Piracaia, Santo Antônio de Posse, Serra Negra, Soco...

Resultado 2:
Documento: EIA-384-24-e-amb-90935-24-Unid-Proc-Oleos-Contam-PCBs-MG-Trafos-Pedreira.pdf
Chunk ID: EIA-384-24-e-amb-90935-24-Unid-Proc-Oleos-Contam-PCBs-MG-Trafos-Pedreira.pdf_chunk_267
Link: https://cetesb.sp.gov.br/eiarima/eia/EIA-384-24-e-amb-90935-24-Unid-Proc-Oleos-Contam-PCBs-MG-Trafos-Pedreira.pdf
Texto: De acordo com o Artigo 1º da Lei 7.438/91: F

In [7]:
# #limpar o banco de dadso caso necessário
# # Conexão com o MongoDB
# from pymongo import MongoClient

# mongo_client = MongoClient("mongodb://localhost:27017")
# mongo_db = mongo_client["rag_db"]
# mongo_chunks = mongo_db["chunks"]

# # Remover todos os documentos da collection
# mongo_chunks.delete_many({})
# print("Coleção 'chunks' do MongoDB limpa com sucesso.")


Coleção 'chunks' do MongoDB limpa com sucesso.


In [8]:
# #limpar o chromadb caso necessário
# import chromadb

# # Inicializar o cliente ChromaDB
# chroma_client = chromadb.PersistentClient(path="./chroma_db")
# collection = chroma_client.get_or_create_collection(name="rag_embeddings")

# # Buscar todos os IDs
# all_ids = collection.get()["ids"]

# # Remover todos os documentos pelo ID
# if all_ids:
#     collection.delete(ids=all_ids)
#     print(f"Removidos {len(all_ids)} documentos da coleção 'rag_embeddings'.")
# else:
#     print("Nenhum documento encontrado para remoção no ChromaDB.")


Removidos 573 documentos da coleção 'rag_embeddings'.


In [15]:

#estudar leitura de imagens (leitura de mapas)
